In [ ]:
import nni
import torch
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import nni.retiarii.nn.pytorch as nn
import nni.retiarii.strategy as strategy

from collections import OrderedDict

from nni.experiment import Experiment
from nni.retiarii import model_wrapper
from nni.retiarii.evaluator import FunctionalEvaluator
from nni.retiarii.experiment.pytorch import RetiariiExperiment, RetiariiExeConfig

from skimage.metrics import peak_signal_noise_ratio as skimage_psnr

from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from torchvision.datasets import MNIST
from torchvision import datasets, transforms




# non search

In [ ]:
class ModelTest(nn.Module):
    def __init__(self, in_channels=3, out_channels=3, hidden_channels=16):
        super(ModelTest, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, hidden_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(hidden_channels)
        self.act1 = nn.LeakyReLU(inplace=True)
        self.conv2 = nn.Conv2d(hidden_channels, out_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.act2 = nn.Sigmoid()
    
    def forward(self, x):
        x = self.act1(self.bn1(self.conv1(x)))
        x = self.act2(self.bn2(self.conv2(x)))
        return x


def add_noise(img, noise_factor=0.5):
    """Add random noise to an image."""
    noise = torch.randn_like(img) * noise_factor
    noisy_img = img + noise
    return torch.clamp(noisy_img, 0., 1.)

def deep_image_prior_denoising(model, noisy_img, device, optimizer, iterations=3000):
    model.train()
    for iteration in range(iterations):
        optimizer.zero_grad()
        output = model(torch.randn(noisy_img.shape).to(device))
        loss = nn.MSELoss()(output, noisy_img)
        loss.backward()
        optimizer.step()
        if iteration % 1000 == 0:
            print('Iteration: {}\tLoss: {:.6f}'.format(iteration, loss.item()))


    return output

def denoise_image():
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    # Load and preprocess the noisy image
    transform = transforms.Compose([transforms.ToTensor()])
    dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    
    # Let's use the first image from the dataset
    img, _ = dataset[0]
    noisy_img = add_noise(img).unsqueeze(0).to(device)  # Add noise and batch dimension

    # Initialize model and optimizer
    model = ModelTest(in_channels=3, out_channels=3).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    # Denoise the image using Deep Image Prior
    denoised_img = deep_image_prior_denoising(model, noisy_img, device, optimizer)

    # Visualize the noisy and denoised images
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1)
    plt.imshow(np.transpose(img.numpy(), (1, 2, 0)))
    plt.title('Original Image')
    
    plt.subplot(1, 3, 2)
    plt.imshow(np.transpose(noisy_img[0].cpu().numpy(), (1, 2, 0)))
    plt.title('Noisy Image')

    plt.subplot(1, 3, 3)
    plt.imshow(np.transpose(denoised_img[0].detach().cpu().numpy(), (1, 2, 0)))
    plt.title('Denoised Image')
    plt.show()

# Call the function
# denoise_image()

In [ ]:
# self.downsample1 = nn.MaxPool2d(kernel_size=2, stride=2)


# self.upsample = nn.Upsample(scale_factor=2,mode='bilinear', align_corners=True)
# self.upsample = nn.Upsample(scale_factor=2,mode='bicubic', align_corners=True)


# expanded model

In [ ]:
class Test(nn.Module):
    def __init__(self, in_channels=3, out_channels=3):
        super().__init__()

        hidden_channels = 64
        ks = 3 # nn.ValueChoice([3, 7])
        dl = 1 # nn.ValueChoice([3, 4])
        pd = (ks-1)*dl//2

        # Conv layer in
        self.conv1 = nn.Conv2d(in_channels, hidden_channels, kernel_size=ks, padding=pd, dilation=dl)
        self.bn1 = nn.BatchNorm2d(hidden_channels)
        self.act1 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(hidden_channels, hidden_channels, kernel_size=ks, padding=pd, dilation=dl)
        self.bn2 = nn.BatchNorm2d(hidden_channels)
        self.act2 = nn.Sigmoid()

        self.first = nn.Sequential(
            self.conv1,self.bn1,self.act1,
            self.conv2, self.bn2,self.act2
        )

        # Encoder 1
        self.downsample1 = nn.AvgPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(hidden_channels, hidden_channels*2, kernel_size=ks, padding=pd, dilation=dl)
        self.bn3 = nn.BatchNorm2d(hidden_channels*2)
        self.act3 = nn.ReLU(inplace=True)
        self.conv4 = nn.Conv2d(hidden_channels*2, hidden_channels*2, kernel_size=ks, padding=pd, dilation=dl)
        self.bn4 = nn.BatchNorm2d(hidden_channels*2)
        self.act4 = nn.Sigmoid()

        self.Encoder1 = nn.Sequential(
            self.downsample1,
            self.conv3,self.bn3,self.act3,
            self.conv4,self.bn4,self.act4
        )


        # Decoder 1
        self.upsample1 = nn.Upsample(scale_factor=2,mode='nearest')

        self.conv5 = nn.Conv2d(hidden_channels*3, hidden_channels, kernel_size=ks, padding=pd, dilation=dl)
        self.bn5 = nn.BatchNorm2d(hidden_channels)
        self.act5 = nn.ReLU(inplace=True)
        self.conv6 = nn.Conv2d(hidden_channels, hidden_channels, kernel_size=ks, padding=pd, dilation=dl)
        self.bn6 = nn.BatchNorm2d(hidden_channels)
        self.act6 = nn.Sigmoid()

        self.Decoder1 = nn.Sequential(
            self.conv5, self.bn5, self.act5,
            self.conv6, self.bn6, self.act6
            )

        # Conv layer out
        self.out = nn.Conv2d(hidden_channels, out_channels, kernel_size=ks, padding=pd, dilation=dl)
    
    def crop_tensor(self, target_tensor, tensor):
        target_size = target_tensor.size()[2]  # Assuming height and width are same
        tensor_size = tensor.size()[2]
        delta = tensor_size - target_size
        delta = delta // 2
        return tensor[:, :, delta:tensor_size-delta, delta:tensor_size-delta]
    
    def upsample_and_crop(self, input, skip, upsample, decode):
        upsampled = upsample(input)
        cropped = self.crop_tensor(upsampled, skip)
        return decode(torch.cat([cropped, upsampled], 1))

    def print_info(self, layer_name, tensor):
        separator = "-" * 50
        if layer_name == "Input":
            print(separator)
            print(separator)
        if layer_name == "Output":
            print(f"Layer: {layer_name} -- Conv Layer Out")
            print(f"Output Shape: {tensor.shape}")
            print(separator)
            print(separator)
        else:        
            print(f"Layer: {layer_name}")
            print(f"Output Shape: {tensor.shape}")
            print(separator)
    
    def forward(self, x):
        self.print_info("Input", x)

        x = self.first(x)
        self.print_info("Conv layer in", x)

        x1 = self.Encoder1(x)
        self.print_info("Encoder 1", x1)

        x2 = self.upsample_and_crop(
            x1, # input
            x, # skip
            self.upsample1, # upsample 
            self.Decoder1 # decode
            )
        self.print_info("Decoder 1", x2)

        x3 = self.out(x2)
        self.print_info("Output", x3)

        return x3

x = torch.randn(1, 3, 64, 64)
model = Test()
output = model(x)



# expanded search

In [ ]:
@model_wrapper
class SearchSpace(nn.Module):
    def __init__(self, in_channels=3, out_channels=3):
        super().__init__()

        hidden_channels = 64
        ks = nn.ValueChoice([1, 3], label="Kernel Size")
        dl = nn.ValueChoice([1, 3, 5], label="Dilation Rate")
        pd = (ks-1)*dl//2

        activations = OrderedDict([
            ("RelU", nn.ReLU(inplace=True)),
            ("LeakyRelU", nn.LeakyReLU(inplace=True)),
            ("Sigmoid", nn.Sigmoid()),
            ("Selu", nn.SELU(inplace=True)),
            ("PreLU", nn.PReLU()),
            ("SiLU", nn.SiLU(inplace=True)),
        ])

        downsample = OrderedDict([
            ("AvgPool2d", nn.AvgPool2d(kernel_size=2, stride=2)),
            ("MaxPool2d", nn.MaxPool2d(kernel_size=2, stride=2)),
        ])

        upsample = OrderedDict([
            ("Nearest", nn.Upsample(scale_factor=2,mode='nearest')),
            ("Bilinear", nn.Upsample(scale_factor=2,mode='bilinear', align_corners=True)),
            ("Bicubic", nn.Upsample(scale_factor=2,mode='bicubic', align_corners=True))
        ])

        # Conv layer in"
        self.layer1_name = "1 - Conv layer in"
        self.conv1 = nn.Conv2d(in_channels, hidden_channels, kernel_size=ks, padding=pd, dilation=dl)
        self.bn1 = nn.BatchNorm2d(hidden_channels)
        self.act1 = nn.LayerChoice(activations, label=f"{self.layer1_name}: Activation Function 1")
        self.conv2 = nn.Conv2d(hidden_channels, hidden_channels, kernel_size=ks, padding=pd, dilation=dl)
        self.bn2 = nn.BatchNorm2d(hidden_channels)
        self.act2 = nn.LayerChoice(activations, label=f"{self.layer1_name}: Activation Function 2")

        self.first = nn.Sequential(
            self.conv1,self.bn1,self.act1,
            self.conv2, self.bn2,self.act2
        )

        # Encoder 1
        self.layer2_name = "2 - Encoder 1"
        self.downsample1 = nn.LayerChoice(downsample,label="First Pooling Technique")

        self.conv3 = nn.Conv2d(hidden_channels, hidden_channels*2, kernel_size=ks, padding=pd, dilation=dl)
        self.bn3 = nn.BatchNorm2d(hidden_channels*2)
        self.act3 = nn.LayerChoice(activations, label=f"{self.layer2_name}: Activation Function 1")
        self.conv4 = nn.Conv2d(hidden_channels*2, hidden_channels*2, kernel_size=ks, padding=pd, dilation=dl)
        self.bn4 = nn.BatchNorm2d(hidden_channels*2)
        self.act4 = nn.LayerChoice(activations, label=f"{self.layer2_name}: Activation Function 2")

        self.Encoder1 = nn.Sequential(
            self.downsample1,
            self.conv3,self.bn3,self.act3,
            self.conv4,self.bn4,self.act4
        )

        # Decoder 1
        self.layer3_name = "3 - Decoder 1"
        self.upsample1 = nn.LayerChoice(upsample, label="First Upsample Technique")

        self.conv5 = nn.Conv2d(hidden_channels*3, hidden_channels, kernel_size=ks, padding=pd, dilation=dl)
        self.bn5 = nn.BatchNorm2d(hidden_channels)
        self.act5 = nn.LayerChoice(activations, label=f"{self.layer3_name}: Activation Function 1")
        self.conv6 = nn.Conv2d(hidden_channels, hidden_channels, kernel_size=ks, padding=pd, dilation=dl)
        self.bn6 = nn.BatchNorm2d(hidden_channels)
        self.act6 = nn.LayerChoice(activations, label=f"{self.layer3_name}: Activation Function 2")

        self.Decoder1 = nn.Sequential(
            self.conv5, self.bn5, self.act5,
            self.conv6, self.bn6, self.act6
            )

        # Conv layer out
        self.out = nn.Conv2d(hidden_channels, out_channels, kernel_size=ks, padding=pd, dilation=dl)
    
    def crop_tensor(self, target_tensor, tensor):
        target_size = target_tensor.size()[2]  # Assuming height and width are same
        tensor_size = tensor.size()[2]
        delta = tensor_size - target_size
        delta = delta // 2
        return tensor[:, :, delta:tensor_size-delta, delta:tensor_size-delta]
    
    def upsample_and_crop(self, input, skip, upsample, decode):
        upsampled = upsample(input)
        cropped = self.crop_tensor(upsampled, skip)
        return decode(torch.cat([cropped, upsampled], 1))

    def print_info(self, layer_name, tensor):
        separator = "-" * 50
        if layer_name == "Input":
            print(separator)
            print(separator)
        if layer_name == "Output":
            print(f"Layer: {layer_name} -- Conv Layer Out")
            print(f"Output Shape: {tensor.shape}")
            print(separator)
            print(separator)
        else:        
            print(f"Layer: {layer_name}")
            print(f"Output Shape: {tensor.shape}")
            print(separator)
    
    def forward(self, x):
        self.print_info("Input", x)

        x = self.first(x)
        self.print_info("Conv layer in", x)

        x1 = self.Encoder1(x)
        self.print_info("Encoder 1", x1)

        x2 = self.upsample_and_crop(
            x1, # input
            x, # skip
            self.upsample1, # upsample 
            self.Decoder1 # decode
            )
        self.print_info("Decoder 1", x2)

        x3 = self.out(x2)
        self.print_info("Output", x3)

        return x3


In [ ]:
def psnr(image_true, image_test):
    # Convert PyTorch tensors to NumPy arrays
    if torch.is_tensor(image_true):
        image_true = image_true.detach().cpu().numpy()
    if torch.is_tensor(image_test):
        image_test = image_test.detach().cpu().numpy()
    return skimage_psnr(image_true, image_test)

def deep_image_prior_denoising(model, noisy_img, clean_img, device, optimizer, iterations=3000):
    model.train()
    for iteration in range(iterations):
        optimizer.zero_grad()
        output = model(torch.randn(noisy_img.shape).to(device))
        loss = nn.MSELoss()(output, noisy_img)
        loss.backward()
        optimizer.step()
        if iteration % 1000 == 0:
            # Calculate PSNR
            with torch.no_grad():
                denoised_output = model(noisy_img)
                psnr_value = psnr(clean_img, denoised_output)
            print('Iteration: {}\tLoss: {:.6f}\tPSNR: {:.6f} dB'.format(iteration, loss.item(), psnr_value))
            nni.report_intermediate_result(psnr_value)
    return output

def evaluate_denoising(denoised_img, clean_img):
    # We no longer need the model in an eval state or any forward pass here
    # because the denoised image is already generated and passed to the function.
    return psnr(clean_img, denoised_img)

def main_evaluation(model_cls):
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    
    # Instantiate model
    model = model_cls().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    transform = transforms.Compose([transforms.ToTensor()])
    dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    img, _ = dataset[0]  # Original clean image
    noisy_img = add_noise(img).unsqueeze(0).to(device)  # Noisy version of image

    # Denoise the image for a set number of iterations
    denoised_img = deep_image_prior_denoising(model, noisy_img, img.unsqueeze(0).to(device), device, optimizer)

    # Evaluate the PSNR of the denoised image
    psnr_value = evaluate_denoising(denoised_img, img.unsqueeze(0).to(device))
    print('PSNR: {:.6f} dB'.format(psnr_value))

    # Report final PSNR to NNI
    nni.report_final_result(psnr_value.item())


In [ ]:
# search space
model_space = SearchSpace()
evaluator = FunctionalEvaluator(main_evaluation)

# search strategy
search_strategy = strategy.Random(dedup=True)

# experiment
exp = RetiariiExperiment(model_space, evaluator, [], search_strategy)
exp_config = RetiariiExeConfig('local')
exp_config.experiment_name = 'mnist_search'
exp_config.trial_code_directory = 'C:/Users/Public/Public_VS_Code/NAS_test'
exp_config.experiment_working_directory = 'C:/Users/Public/nni-experiments'

exp_config.max_trial_number = 24   # spawn 4 trials at most
exp_config.trial_concurrency = 2  # will run two trials concurrently

exp_config.trial_gpu_number = 1
exp_config.training_service.use_active_gpu = True

# Execute
exp.run(exp_config, 8081)

In [ ]:
experiment = Experiment.connect(8081)
experiment.stop()

# search

In [ ]:
class DepthwiseSeparableConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.depthwise = nn.Conv2d(in_ch, in_ch, kernel_size=3, groups=in_ch)
        self.pointwise = nn.Conv2d(in_ch, out_ch, kernel_size=1)

    def forward(self, x):
        return self.pointwise(self.depthwise(x))

@model_wrapper
class ModelSpace(nn.Module):
    def __init__(self, in_channels=3, out_channels=3):
        super().__init__()

        hidden_channels = 16
        ks = nn.ValueChoice([3, 7])
        dl = nn.ValueChoice([3, 4])
        pd = (ks-1)*dl//2

        self.conv1 = nn.Conv2d(in_channels, hidden_channels, kernel_size=ks, padding=pd, dilation=dl)
        self.bn1 = nn.BatchNorm2d(hidden_channels)
        self.act1 = nn.LayerChoice([
                        nn.ReLU(inplace=True),
                        # nn.LeakyReLU(inplace=True),
                        # nn.PReLU(),
                        # nn.SELU(),
                        # nn.SiLU(inplace=True),
                    ])
        self.conv2 = nn.Conv2d(hidden_channels, hidden_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(hidden_channels)
        self.act2 = nn.LayerChoice([
                        nn.Sigmoid(),
                    ])
        
        self.downsample = nn.MaxPool2d(2)

        self.conv3 = nn.Conv2d(hidden_channels, hidden_channels*2, kernel_size=ks, padding=pd, dilation=dl)
        self.bn3 = nn.BatchNorm2d(hidden_channels*2)
        self.act3 = nn.LayerChoice([
                        nn.ReLU(inplace=True),
                    ])
        self.conv4 = nn.Conv2d(hidden_channels*2, hidden_channels*2, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(hidden_channels*2)
        self.act4 = nn.LayerChoice([
                        nn.Sigmoid(),
                    ])

        self.upsample = nn.Upsample(scale_factor=2)

        self.conv3 = nn.Conv2d(hidden_channels, out_channels, kernel_size=ks, padding=pd, dilation=dl)
        self.bn3 = nn.BatchNorm2d(out_channels)
        self.act3 = nn.LayerChoice([
                        nn.ReLU(inplace=True),
                    ])
        self.conv4 = nn.Conv2d(out_channels, out_channels, kernel_size=ks, padding=pd, dilation=dl)
        self.bn4 = nn.BatchNorm2d(out_channels)
        self.act4 = nn.LayerChoice([
                        nn.Sigmoid(),
                    ])
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.act1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.act2(x)
        return x

def psnr(image_true, image_test):
    # Convert PyTorch tensors to NumPy arrays
    if torch.is_tensor(image_true):
        image_true = image_true.detach().cpu().numpy()
    if torch.is_tensor(image_test):
        image_test = image_test.detach().cpu().numpy()
    return skimage_psnr(image_true, image_test)

def deep_image_prior_denoising(model, noisy_img, clean_img, device, optimizer, iterations=3000):
    model.train()
    for iteration in range(iterations):
        optimizer.zero_grad()
        output = model(torch.randn(noisy_img.shape).to(device))
        loss = nn.MSELoss()(output, noisy_img)
        loss.backward()
        optimizer.step()
        if iteration % 1000 == 0:
            # Calculate PSNR
            with torch.no_grad():
                denoised_output = model(noisy_img)
                psnr_value = psnr(clean_img, denoised_output)
            print('Iteration: {}\tLoss: {:.6f}\tPSNR: {:.6f} dB'.format(iteration, loss.item(), psnr_value))
            nni.report_intermediate_result(psnr_value)
    return output

def evaluate_denoising(denoised_img, clean_img):
    # We no longer need the model in an eval state or any forward pass here
    # because the denoised image is already generated and passed to the function.
    return psnr(clean_img, denoised_img)

def main_evaluation(model_cls):
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    
    # Instantiate model
    model = model_cls().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    transform = transforms.Compose([transforms.ToTensor()])
    dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    img, _ = dataset[0]  # Original clean image
    noisy_img = add_noise(img).unsqueeze(0).to(device)  # Noisy version of image

    # Denoise the image for a set number of iterations
    denoised_img = deep_image_prior_denoising(model, noisy_img, img.unsqueeze(0).to(device), device, optimizer)

    # Evaluate the PSNR of the denoised image
    psnr_value = evaluate_denoising(denoised_img, img.unsqueeze(0).to(device))
    print('PSNR: {:.6f} dB'.format(psnr_value))

    # Report final PSNR to NNI
    nni.report_final_result(psnr_value.item())



In [ ]:
# search space
model_space = ModelSpace()
evaluator = FunctionalEvaluator(main_evaluation)

# search strategy
search_strategy = strategy.Random(dedup=True)

# experiment
exp = RetiariiExperiment(model_space, evaluator, [], search_strategy)
exp_config = RetiariiExeConfig('local')
exp_config.experiment_name = 'mnist_search'
exp_config.trial_code_directory = 'C:/Users/Public/Public_VS_Code/NAS_test'
exp_config.experiment_working_directory = 'C:/Users/Public/nni-experiments'

exp_config.max_trial_number = 24   # spawn 4 trials at most
exp_config.trial_concurrency = 2  # will run two trials concurrently

exp_config.trial_gpu_number = 1
exp_config.training_service.use_active_gpu = True

# Execute
exp.run(exp_config, 8081)

In [ ]:
experiment = Experiment.connect(8081)
experiment.stop()